## Step 1: Environment Setup & GPU Check

In [ ]:
# Check GPU availability
!nvidia-smi

# Verify T4 GPU
import torch
print(f"\nPyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install required packages
!pip install -q yfinance xgboost scikit-learn pandas numpy python-dotenv requests

print("Packages installed successfully")

## Step 2: Mount Google Drive (Save Models Here)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create workspace
import os
workspace = '/content/drive/MyDrive/underdog_trader'
os.makedirs(workspace, exist_ok=True)
os.makedirs(f'{workspace}/models', exist_ok=True)
os.makedirs(f'{workspace}/data', exist_ok=True)

print(f"Workspace created: {workspace}")

## Step 3: Clone Repository & Import Modules

In [ ]:
# Clone repo (or upload files manually)
!git clone https://github.com/alexpayne556-collab/quantum-ai-trader_v1.1.git /content/quantum-ai-trader

# Add to path
import sys
sys.path.append('/content/quantum-ai-trader/src/python')

print("Repository cloned successfully")

In [ ]:
# Import our modules
from multi_model_ensemble import MultiModelEnsemble
from feature_engine import FeatureEngine
from regime_classifier import RegimeClassifier

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import logging

logging.basicConfig(level=logging.INFO)
print("Modules imported successfully")

## Step 4: Define Alpha 76 Watchlist

In [ ]:
# Alpha 76 high-velocity small/mid-cap stocks
ALPHA_76 = [
    # Autonomous & AI Hardware
    'SYM', 'IONQ', 'RGTI', 'QUBT', 'AMBA', 'LAZR', 'INVZ', 'OUST', 'AEVA', 'SERV',
    
    # Space Economy
    'RKLB', 'ASTS', 'LUNR', 'JOBY', 'ACHR', 'PL', 'SPIR', 'IRDM',
    
    # Biotech (Gene Editing & Rare Disease)
    'VKTX', 'NTLA', 'BEAM', 'CRSP', 'EDIT', 'VERV', 'BLUE', 'FATE', 'AKRO', 'KOD',
    'CYTK', 'LEGN', 'RARE', 'SRPT', 'BMRN', 'ALNY',
    
    # Green Energy & Grid
    'FLNC', 'NXT', 'BE', 'ARRY', 'ENPH', 'ENOV', 'QS', 'VST', 'AES',
    
    # Fintech & Digital Assets
    'SOFI', 'COIN', 'HOOD', 'UPST', 'AFRM', 'LC', 'MARA', 'SQ', 'NU',
    
    # Next-Gen Consumer & Software
    'APP', 'DUOL', 'PATH', 'S', 'CELH', 'ONON', 'SOUN', 'FOUR', 'NET', 'GTLB',
    'DDOG', 'SNOW', 'PLTR', 'RBLX', 'U'
]

print(f"Alpha 76 Watchlist: {len(ALPHA_76)} tickers")
print(f"Sectors: Autonomous, Space, Biotech, Energy, Fintech, Software")

## Step 5: Download Historical Data (2 Years, 1hr Bars)

In [ ]:
def download_ticker_data(ticker: str, period: str = '2y', interval: str = '1h') -> pd.DataFrame:
    """Download OHLCV data for single ticker"""
    try:
        df = yf.download(ticker, period=period, interval=interval, progress=False)
        if len(df) > 0:
            df['ticker'] = ticker
            df = df.reset_index()
            df.columns = [c.lower() for c in df.columns]
            return df
    except Exception as e:
        print(f"Error downloading {ticker}: {e}")
    return pd.DataFrame()

print("Downloading data for Alpha 76 (this takes 10-15 minutes)...")

all_data = []
failed_tickers = []

for i, ticker in enumerate(ALPHA_76):
    if (i + 1) % 10 == 0:
        print(f"Progress: {i+1}/{len(ALPHA_76)} tickers")
    
    df = download_ticker_data(ticker)
    if len(df) > 100:  # Need minimum data
        all_data.append(df)
    else:
        failed_tickers.append(ticker)

print(f"\nDownloaded: {len(all_data)} tickers")
print(f"Failed: {len(failed_tickers)} tickers: {failed_tickers}")

# Combine all data
raw_data = pd.concat(all_data, ignore_index=True)
print(f"\nTotal rows: {len(raw_data):,}")
print(f"Date range: {raw_data['datetime'].min()} to {raw_data['datetime'].max()}")

# Save raw data
raw_data.to_csv(f'{workspace}/data/alpha_76_raw.csv', index=False)
print(f"\nSaved to: {workspace}/data/alpha_76_raw.csv")

## Step 6: Calculate Features (30+ Indicators)

In [ ]:
print("Calculating features for each ticker...")

engine = FeatureEngine()
feature_data = []

for ticker in raw_data['ticker'].unique():
    print(f"Processing {ticker}...")
    
    # Get ticker data
    ticker_df = raw_data[raw_data['ticker'] == ticker].copy()
    ticker_df = ticker_df.rename(columns={'datetime': 'timestamp'})
    
    # Calculate features
    try:
        df_features = engine.calculate_all_features(ticker_df)
        df_features = engine.fill_missing_values(df_features)
        df_features['ticker'] = ticker
        feature_data.append(df_features)
    except Exception as e:
        print(f"  Error: {e}")

# Combine
features_df = pd.concat(feature_data, ignore_index=True)
print(f"\nFeatures calculated: {len(engine.get_feature_names())} features")
print(f"Total rows: {len(features_df):,}")

# Save
features_df.to_csv(f'{workspace}/data/alpha_76_features.csv', index=False)
print(f"Saved to: {workspace}/data/alpha_76_features.csv")

## Step 7: Prepare Training Data (Labels + Split)

In [ ]:
print("Preparing training data...")

# Initialize ensemble to use label creation
ensemble = MultiModelEnsemble(use_gpu=True)

# Create labels for each ticker
labeled_data = []

for ticker in features_df['ticker'].unique():
    ticker_df = features_df[features_df['ticker'] == ticker].copy()
    
    # Create labels (5-bar forward return classification)
    labels = ensemble.prepare_labels(ticker_df['close'], forward_periods=5)
    ticker_df['label'] = labels
    
    labeled_data.append(ticker_df)

training_data = pd.concat(labeled_data, ignore_index=True)

# Remove rows with no label (last 5 bars of each ticker)
training_data = training_data.dropna(subset=['label'])

print(f"Total training samples: {len(training_data):,}")

# Check label distribution
label_counts = training_data['label'].value_counts().sort_index()
print(f"\nLabel distribution:")
print(f"  SELL (0): {label_counts.get(0, 0):,} ({label_counts.get(0, 0)/len(training_data)*100:.1f}%)")
print(f"  HOLD (1): {label_counts.get(1, 0):,} ({label_counts.get(1, 0)/len(training_data)*100:.1f}%)")
print(f"  BUY  (2): {label_counts.get(2, 0):,} ({label_counts.get(2, 0)/len(training_data)*100:.1f}%)")

In [ ]:
# Train/Validation Split (time-based)
# Use last 20% of data for validation (most recent)

split_idx = int(0.8 * len(training_data))

train_data = training_data.iloc[:split_idx]
val_data = training_data.iloc[split_idx:]

print(f"Train samples: {len(train_data):,}")
print(f"Validation samples: {len(val_data):,}")

# Prepare feature matrices
feature_cols = engine.get_feature_names()

X_train = train_data[feature_cols]
y_train = train_data['label'].values

X_val = val_data[feature_cols]
y_val = val_data['label'].values

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"Features: {len(feature_cols)}")

## Step 8: Train 3-Model Ensemble (GPU Accelerated)

In [ ]:
print("Training Multi-Model Ensemble...")
print("This will take 30-60 minutes on T4 GPU\n")

# Train ensemble
metrics = ensemble.train(X_train, y_train, X_val, y_val)

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

# Display metrics
for model_name, model_metrics in metrics.items():
    print(f"\n{model_name.upper()}:")
    for metric_name, value in model_metrics.items():
        if metric_name != 'error':
            print(f"  {metric_name}: {value:.4f}")
        else:
            print(f"  ERROR: {value}")

print("\n" + "="*60)

## Step 9: Test Ensemble Predictions

In [ ]:
# Test on validation set samples
print("Testing ensemble predictions...\n")

test_samples = 10
test_indices = np.random.choice(len(X_val), test_samples, replace=False)

for idx in test_indices:
    X_test = X_val.iloc[idx:idx+1]
    y_true = y_val[idx]
    
    pred = ensemble.predict(X_test)
    
    true_label = ['SELL', 'HOLD', 'BUY'][int(y_true)]
    
    print(f"Sample {idx}:")
    print(f"  True: {true_label}")
    print(f"  Predicted: {pred['signal']}")
    print(f"  Confidence: {pred['confidence']:.3f}")
    print(f"  Agreement: {pred['agreement']:.3f}")
    print(f"  Votes: {pred['votes']}")
    print()

## Step 10: Save Trained Models to Google Drive

In [ ]:
# Save ensemble
model_path = f'{workspace}/models/ensemble_alpha76_v1'
ensemble.save(model_path)

print(f"Models saved to: {model_path}")
print(f"\nFiles saved:")
!ls -lh {model_path}

# Save training metrics
import json
with open(f'{model_path}/training_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nTraining metrics saved")

## Step 11: Backtest on Recent Data (2024)

In [ ]:
# Simple backtest: Generate signals on validation set
print("Running backtest on validation set...\n")

# Get predictions for entire validation set
val_predictions = ensemble._predict_batch(X_val)

# Add to validation data
val_results = val_data.copy()
val_results['pred_signal'] = val_predictions['signal']
val_results['pred_confidence'] = val_predictions['confidence']

# Calculate forward returns (5 bars ahead)
val_results['forward_return'] = val_results.groupby('ticker')['close'].pct_change(5).shift(-5)

# Filter for high-confidence BUY signals (>0.75)
buy_signals = val_results[
    (val_results['pred_signal'] == 'BUY') & 
    (val_results['pred_confidence'] > 0.75)
].copy()

print(f"High-confidence BUY signals: {len(buy_signals)}")

if len(buy_signals) > 0:
    # Calculate win rate
    buy_signals = buy_signals.dropna(subset=['forward_return'])
    win_rate = (buy_signals['forward_return'] > 0).sum() / len(buy_signals)
    avg_return = buy_signals['forward_return'].mean()
    
    print(f"\nBacktest Results (High-Confidence BUY):")
    print(f"  Win Rate: {win_rate:.2%}")
    print(f"  Avg Return: {avg_return:.2%}")
    print(f"  Total Signals: {len(buy_signals)}")
    
    # Best trades
    print(f"\nTop 10 Trades:")
    top_trades = buy_signals.nlargest(10, 'forward_return')[['ticker', 'timestamp', 'close', 'forward_return', 'pred_confidence']]
    print(top_trades.to_string(index=False))

## Step 12: Export for Production Use

In [ ]:
# Create deployment package
import shutil

deploy_path = f'{workspace}/deploy'
os.makedirs(deploy_path, exist_ok=True)

# Copy models
shutil.copytree(model_path, f'{deploy_path}/models', dirs_exist_ok=True)

# Copy modules
shutil.copy('/content/quantum-ai-trader/src/python/multi_model_ensemble.py', deploy_path)
shutil.copy('/content/quantum-ai-trader/src/python/feature_engine.py', deploy_path)
shutil.copy('/content/quantum-ai-trader/src/python/regime_classifier.py', deploy_path)

# Create README
readme = f"""
# Underdog Trading System - Trained Models

Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Hardware: Colab Pro T4 GPU
Training Data: {len(train_data):,} samples
Validation Data: {len(val_data):,} samples

## Model Performance
{json.dumps(metrics, indent=2)}

## Usage
```python
from multi_model_ensemble import MultiModelEnsemble

ensemble = MultiModelEnsemble()
ensemble.load('models/ensemble_alpha76_v1')

# Make prediction
prediction = ensemble.predict(features_df)
```

## Files
- models/ - Trained ensemble (XGBoost, RF, GB)
- multi_model_ensemble.py - Ensemble class
- feature_engine.py - Feature calculation
- regime_classifier.py - Market regime detection
"""

with open(f'{deploy_path}/README.md', 'w') as f:
    f.write(readme)

print(f"Deployment package ready: {deploy_path}")
print(f"\nDownload this folder to your local machine for production use")
print(f"\nFiles:")
!ls -lh {deploy_path}

## TRAINING COMPLETE!

### Next Steps:
1. Download the `/content/drive/MyDrive/underdog_trader/deploy` folder
2. Copy to your local `quantum-ai-trader_v1.1` repository
3. Build the live trading engine to use these models
4. Paper trade for 1 week before going live

### Your Advantage:
- 3-model ensemble trained on 1.3M+ data points
- GPU-accelerated XGBoost for speed
- 30+ features per ticker
- Market regime awareness
- Alpha 76 high-velocity watchlist

**Intelligence edge, not speed edge**